# Session 7: Evaluation & Testing

**Duration**: 75 minutes  
**Difficulty**: Intermediate

## Learning Objectives

- 🎯 Create test datasets
- 🎯 Define quality metrics
- 🎯 Build evaluation frameworks
- 🎯 Implement automated testing
- 🎯 A/B test different approaches
- 🎯 Set up monitoring dashboards

## 📚 Why Evaluate?

**Problems without evaluation**:
- Don't know if system is improving
- Can't compare approaches
- Don't catch regressions
- No data for decisions

**Benefits of evaluation**:
- Measure quality objectively
- Track improvements over time
- Compare models/prompts
- Catch bugs early

## Part 0: Setup

In [ ]:
# Install required packages
!pip install openai python-dotenv pandas matplotlib -q

print("✅ Packages installed!")

In [ ]:
import os
import json
import random
import time
import pandas as pd
import matplotlib.pyplot as plt
from openai import OpenAI
from typing import List, Dict

# Set up API key
try:
    from google.colab import userdata
    os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
    print("✅ API key loaded from Colab secrets")
except:
    from getpass import getpass
    if 'OPENAI_API_KEY' not in os.environ:
        os.environ['OPENAI_API_KEY'] = getpass('Enter your OpenAI API key: ')
    print("✅ API key loaded")

# Initialize client
client = OpenAI(api_key=os.environ.get('OPENAI_API_KEY'))

print("\n🚀 Ready to build evaluation systems!")

## Part 1: Creating Test Datasets

A good test dataset is the foundation of evaluation.

In [ ]:
# Create a golden test dataset

test_cases = [
    {
        "id": 1,
        "category": "order_status",
        "query": "Where is my order ORD-12345?",
        "expected_function": "get_order_status",
        "expected_args": {"order_id": "ORD-12345"},
        "expected_contains": ["shipped", "tracking"],
        "difficulty": "easy"
    },
    {
        "id": 2,
        "category": "refund",
        "query": "I want a refund for my broken product",
        "expected_function": "create_support_ticket",
        "expected_contains": ["ticket", "refund"],
        "difficulty": "medium"
    },
    {
        "id": 3,
        "category": "policy",
        "query": "What is your return policy?",
        "expected_function": "search_knowledge_base",
        "expected_contains": ["30 days", "return"],
        "difficulty": "easy"
    },
    {
        "id": 4,
        "category": "technical",
        "query": "My laptop won't turn on and the light is blinking",
        "expected_function": "create_support_ticket",
        "expected_contains": ["troubleshooting", "technical"],
        "difficulty": "hard"
    },
    {
        "id": 5,
        "category": "account",
        "query": "Check my rewards points for john@email.com",
        "expected_function": "check_account_info",
        "expected_args": {"customer_email": "john@email.com"},
        "expected_contains": ["points", "account"],
        "difficulty": "easy"
    }
]

print(f"✅ Created test dataset with {len(test_cases)} test cases\n")

# Display test cases
df = pd.DataFrame(test_cases)
print(df[['id', 'category', 'query', 'difficulty']])

## Part 2: Automated Evaluation Framework

In [ ]:
# Simple mock system for testing

def mock_rag_system(query: str) -> str:
    """Mock RAG system for testing"""
    
    # Simple keyword-based responses
    if "return policy" in query.lower():
        return "Our return policy allows returns within 30 days of purchase."
    elif "order" in query.lower() and "ORD" in query:
        return "Your order has been shipped. Tracking number: 1Z999AA."
    elif "refund" in query.lower():
        return "We've created ticket TKT-12345 for your refund request."
    elif "laptop" in query.lower() and "won't turn on" in query.lower():
        return "Let's troubleshoot your laptop. Try holding the power button for 10 seconds."
    elif "rewards" in query.lower() or "points" in query.lower():
        return "Your account has 450 rewards points."
    else:
        return "I'm not sure how to help with that. Let me connect you with support."

# Test it
print("Testing mock system:\n")
test_query = "What is your return policy?"
response = mock_rag_system(test_query)
print(f"Query: {test_query}")
print(f"Response: {response}")

In [ ]:
class RAGEvaluator:
    """Evaluate RAG system quality"""
    
    def __init__(self, system):
        self.system = system
    
    def evaluate_responses(self, test_cases: List[Dict]) -> Dict:
        """Evaluate system responses against test cases"""
        
        results = []
        
        for test in test_cases:
            # Get system response
            response = self.system(test['query'])
            
            # Check if expected keywords are in response
            contains_expected = all(
                keyword.lower() in response.lower()
                for keyword in test.get('expected_contains', [])
            )
            
            # Calculate score
            score = 1.0 if contains_expected else 0.0
            
            results.append({
                "test_id": test['id'],
                "query": test['query'],
                "response": response,
                "contains_expected": contains_expected,
                "score": score,
                "category": test['category'],
                "difficulty": test['difficulty']
            })
        
        # Calculate aggregate metrics
        total_score = sum(r['score'] for r in results)
        accuracy = total_score / len(results) if results else 0
        
        return {
            "accuracy": accuracy,
            "total_tests": len(results),
            "passed": int(total_score),
            "failed": len(results) - int(total_score),
            "details": results
        }

# Run evaluation
evaluator = RAGEvaluator(mock_rag_system)
eval_results = evaluator.evaluate_responses(test_cases)

print("📊 Evaluation Results:\n")
print(f"Accuracy: {eval_results['accuracy']:.1%}")
print(f"Passed: {eval_results['passed']}/{eval_results['total_tests']}")
print(f"Failed: {eval_results['failed']}/{eval_results['total_tests']}")

# Show failures
print("\n❌ Failed Tests:")
for result in eval_results['details']:
    if result['score'] == 0:
        print(f"  - Test {result['test_id']}: {result['query']}")

## Part 3: LLM-as-Judge

Use LLMs to evaluate other LLM outputs.

In [ ]:
def llm_evaluate(query: str, response: str, expected_elements: List[str]) -> Dict:
    """
    Use GPT-4 to evaluate a response
    """
    
    eval_prompt = f"""Evaluate this customer support response.

Query: {query}
Response: {response}
Expected elements: {', '.join(expected_elements)}

Rate from 1-5 on:
1. Accuracy (Does it answer correctly?)
2. Helpfulness (Is it useful to the customer?)
3. Tone (Is it professional and empathetic?)
4. Completeness (Does it address all aspects?)

Return JSON:
{{
  "accuracy": 1-5,
  "helpfulness": 1-5,
  "tone": 1-5,
  "completeness": 1-5,
  "overall_score": 1-5,
  "reasoning": "brief explanation"
}}"""
    
    result = client.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "user", "content": eval_prompt}],
        response_format={"type": "json_object"}
    )
    
    return json.loads(result.choices[0].message.content)

# Test LLM-as-judge
test_case = test_cases[0]
response = mock_rag_system(test_case['query'])

print(f"Query: {test_case['query']}")
print(f"Response: {response}\n")

evaluation = llm_evaluate(
    test_case['query'],
    response,
    test_case['expected_contains']
)

print("🤖 LLM Evaluation:")
print(json.dumps(evaluation, indent=2))

In [ ]:
# Run LLM-as-judge on all test cases

print("Running LLM-as-Judge evaluation...\n")

llm_eval_results = []

for test in test_cases:
    response = mock_rag_system(test['query'])
    
    evaluation = llm_evaluate(
        test['query'],
        response,
        test.get('expected_contains', [])
    )
    
    llm_eval_results.append({
        "test_id": test['id'],
        "category": test['category'],
        **evaluation
    })
    
    print(f"Test {test['id']}: Overall Score = {evaluation['overall_score']}/5")

# Calculate average scores
avg_accuracy = sum(r['accuracy'] for r in llm_eval_results) / len(llm_eval_results)
avg_overall = sum(r['overall_score'] for r in llm_eval_results) / len(llm_eval_results)

print(f"\n📊 Summary:")
print(f"Average Accuracy: {avg_accuracy:.1f}/5")
print(f"Average Overall: {avg_overall:.1f}/5")

## Part 4: Metrics Dashboard

In [ ]:
class MetricsDashboard:
    """Track system metrics over time"""
    
    def __init__(self):
        self.metrics = {
            "total_queries": 0,
            "successful_queries": 0,
            "average_latency": 0,
            "total_cost": 0,
            "error_count": 0
        }
        self.query_log = []
    
    def track_query(self, query: str, response: str, latency: float, cost: float, error: str = None):
        """Track a single query"""
        
        self.metrics["total_queries"] += 1
        
        if not error:
            self.metrics["successful_queries"] += 1
        else:
            self.metrics["error_count"] += 1
        
        # Update averages
        n = self.metrics["total_queries"]
        self.metrics["average_latency"] = (
            (self.metrics["average_latency"] * (n-1) + latency) / n
        )
        self.metrics["total_cost"] += cost
        
        # Log query
        self.query_log.append({
            "query": query,
            "response": response,
            "latency": latency,
            "cost": cost,
            "error": error,
            "timestamp": time.time()
        })
    
    def get_summary(self) -> Dict:
        """Get metrics summary"""
        total = self.metrics["total_queries"]
        
        return {
            "success_rate": self.metrics["successful_queries"] / total if total > 0 else 0,
            "avg_latency_ms": self.metrics["average_latency"] * 1000,
            "total_cost_usd": self.metrics["total_cost"],
            "error_rate": self.metrics["error_count"] / total if total > 0 else 0,
            "total_queries": total
        }
    
    def plot_metrics(self):
        """Visualize metrics"""
        summary = self.get_summary()
        
        fig, axes = plt.subplots(2, 2, figsize=(12, 8))
        fig.suptitle('System Metrics Dashboard', fontsize=16)
        
        # Success rate
        axes[0, 0].bar(['Success', 'Error'], 
                       [summary['success_rate'], summary['error_rate']])
        axes[0, 0].set_title('Success Rate')
        axes[0, 0].set_ylim([0, 1])
        
        # Latency
        axes[0, 1].bar(['Avg Latency'], [summary['avg_latency_ms']])
        axes[0, 1].set_title('Average Latency (ms)')
        
        # Cost
        axes[1, 0].bar(['Total Cost'], [summary['total_cost_usd']])
        axes[1, 0].set_title('Total Cost (USD)')
        
        # Query count
        axes[1, 1].bar(['Total Queries'], [summary['total_queries']])
        axes[1, 1].set_title('Total Queries')
        
        plt.tight_layout()
        plt.show()

# Test metrics dashboard
dashboard = MetricsDashboard()

# Simulate some queries
for test in test_cases:
    start = time.time()
    response = mock_rag_system(test['query'])
    latency = time.time() - start
    
    dashboard.track_query(
        query=test['query'],
        response=response,
        latency=latency,
        cost=0.002  # Mock cost
    )

# Display summary
summary = dashboard.get_summary()
print("📊 Metrics Summary:\n")
print(f"Success Rate: {summary['success_rate']:.1%}")
print(f"Average Latency: {summary['avg_latency_ms']:.2f}ms")
print(f"Total Cost: ${summary['total_cost_usd']:.4f}")
print(f"Error Rate: {summary['error_rate']:.1%}")
print(f"Total Queries: {summary['total_queries']}")

# Plot
dashboard.plot_metrics()

## Part 5: A/B Testing

In [ ]:
class ABTest:
    """A/B test two different systems"""
    
    def __init__(self, version_a, version_b):
        self.version_a = version_a
        self.version_b = version_b
        self.results = {"a": [], "b": []}
    
    def run_test(self, test_cases: List[Dict], evaluator):
        """Run A/B test"""
        
        for test in test_cases:
            # Randomly assign to A or B
            version = random.choice(["a", "b"])
            
            if version == "a":
                response = self.version_a(test['query'])
            else:
                response = self.version_b(test['query'])
            
            # Evaluate response
            score = evaluator(test, response)
            
            self.results[version].append({
                "test_id": test['id'],
                "score": score,
                "query": test['query']
            })
    
    def analyze(self) -> Dict:
        """Analyze A/B test results"""
        
        avg_a = sum(r['score'] for r in self.results["a"]) / len(self.results["a"]) if self.results["a"] else 0
        avg_b = sum(r['score'] for r in self.results["b"]) / len(self.results["b"]) if self.results["b"] else 0
        
        return {
            "version_a_score": avg_a,
            "version_b_score": avg_b,
            "version_a_count": len(self.results["a"]),
            "version_b_count": len(self.results["b"]),
            "winner": "A" if avg_a > avg_b else "B" if avg_b > avg_a else "Tie",
            "improvement": abs(avg_a - avg_b)
        }

# Create two versions
def version_a(query: str) -> str:
    """Original version"""
    return mock_rag_system(query)

def version_b(query: str) -> str:
    """Improved version with more empathy"""
    response = mock_rag_system(query)
    return f"Thank you for reaching out! {response} Is there anything else I can help you with?"

# Simple evaluator
def simple_evaluator(test: Dict, response: str) -> float:
    """Simple keyword-based scoring"""
    score = 0.0
    for keyword in test.get('expected_contains', []):
        if keyword.lower() in response.lower():
            score += 1.0
    return score / len(test.get('expected_contains', [1]))

# Run A/B test
ab_test = ABTest(version_a, version_b)
ab_test.run_test(test_cases, simple_evaluator)

# Analyze results
results = ab_test.analyze()

print("🔬 A/B Test Results:\n")
print(f"Version A Score: {results['version_a_score']:.2f} ({results['version_a_count']} tests)")
print(f"Version B Score: {results['version_b_score']:.2f} ({results['version_b_count']} tests)")
print(f"\n🏆 Winner: Version {results['winner']}")
print(f"Improvement: {results['improvement']:.2f}")

## Exercises

### Exercise 1: Expand Test Dataset

Create 20 more test cases covering edge cases.

In [ ]:
# Your code here
expanded_test_cases = [
    # TODO: Add 20 more test cases
]

### Exercise 2: Add Response Time Monitoring

Track and visualize response times over time.

In [ ]:
# Your code here
def track_response_times():
    """Track and plot response times"""
    # TODO: Implement response time tracking
    pass

### Exercise 3: Build Regression Test Suite

Create tests that run automatically to catch regressions.

In [ ]:
# Your code here
class RegressionTestSuite:
    """Automated regression testing"""
    # TODO: Implement regression testing
    pass

## 🎉 Session 7 Complete!

### What You Learned:

✅ Evaluation enables improvement  
✅ Create comprehensive test datasets  
✅ Use multiple metrics  
✅ Automate evaluation  
✅ A/B test changes  
✅ LLM-as-judge for quality assessment  
✅ Metrics dashboards for monitoring

### Next Steps:

In **Session 8: Production Deployment**, you'll learn:
- FastAPI application structure
- Redis caching
- Monitoring and logging
- Security and rate limiting
- Docker deployment
- Cloud deployment options

---

**Continue to**: [Session 8: Production →](08_Production.ipynb)